# DKW Controller Evaluation

This notebook evaluates the performance of a proposed method against a baseline for the DKW (Decision-Knowledge-Workflow) Controller. The evaluation focuses on:

- **Decision patterns**: Fusion vs Fission rates
- **Error rates**: Accuracy of predictions
- **API efficiency**: Number of API calls required
- **Performance improvement**: Comparison between methods

In [ ]:
"""Evaluation script for DKW Controller."""
import json
import numpy as np

## Sample Data

Since this is a self-contained notebook, we'll use sample data that represents the results from both baseline and proposed methods. Each method's results contain predictions with decision types ("fusion" or "fission") and error indicators.

In [ ]:
# Sample data that matches the expected evaluation results
# Creating 200 examples to match the statistics from eval_out.json

# Baseline method: 100% fission decisions, 8% error rate
baseline_results = []
for i in range(200):
    error = i < 16  # First 16 examples have errors (8% error rate)
    baseline_results.append({
        "decision": "fission",  # All baseline decisions are fission
        "error": error
    })

# Proposed method: 65% fusion, 35% fission, 9% error rate
proposed_results = []
for i in range(200):
    if i < 130:  # First 130 are fusion (65%)
        decision = "fusion"
    else:  # Remaining 70 are fission (35%)
        decision = "fission"
    
    error = i < 18  # First 18 examples have errors (9% error rate)
    proposed_results.append({
        "decision": decision,
        "error": error
    })

# Combine into the expected format
results = {
    "baseline": baseline_results,
    "proposed": proposed_results
}

print(f"Created sample data:")
print(f"Baseline: {len(baseline_results)} predictions")
print(f"Proposed: {len(proposed_results)} predictions")

## Evaluation Function

The `compute_metrics` function calculates key performance indicators for both methods:

- **Fusion/Fission rates**: Distribution of decision types
- **Error rate**: Percentage of incorrect predictions
- **API calls**: Total and average API usage (fusion = 1 call, fission = 2 calls)
- **Improvement metrics**: Comparison between baseline and proposed methods

In [ ]:
def compute_metrics(results: dict) -> dict:
    """Compute evaluation metrics."""
    metrics = {}

    for method in ["baseline", "proposed"]:
        preds = results[method]

        # Count decisions
        fusion_count = sum(1 for p in preds if p["decision"] == "fusion")
        fission_count = sum(1 for p in preds if p["decision"] == "fission")

        # Compute error rate
        errors = sum(1 for p in preds if p["error"])
        error_rate = errors / len(preds)

        # API calls (fusion=1, fission=2)
        api_calls = fusion_count + 2 * fission_count

        metrics[method] = {
            "fusion_rate": fusion_count / len(preds),
            "fission_rate": fission_count / len(preds),
            "error_rate": error_rate,
            "api_calls": api_calls,
            "avg_calls_per_example": api_calls / len(preds),
        }

    # Compute improvement
    baseline_calls = metrics["baseline"]["avg_calls_per_example"]
    proposed_calls = metrics["proposed"]["avg_calls_per_example"]
    metrics["improvement"] = {
        "api_reduction_pct": (baseline_calls - proposed_calls) / baseline_calls * 100,
        "error_rate_diff": metrics["proposed"]["error_rate"] - metrics["baseline"]["error_rate"],
    }

    return metrics

## Compute Evaluation Metrics

Now let's run the evaluation on our sample data and display the results:

In [ ]:
# Compute evaluation metrics
metrics = compute_metrics(results)

# Display results in a readable format
print("=== EVALUATION RESULTS ===")

print("\nBASELINE METHOD:")
print(f"  Fusion rate:     {metrics['baseline']['fusion_rate']:.1%}")
print(f"  Fission rate:    {metrics['baseline']['fission_rate']:.1%}")
print(f"  Error rate:      {metrics['baseline']['error_rate']:.1%}")
print(f"  Total API calls: {metrics['baseline']['api_calls']}")
print(f"  Avg calls/example: {metrics['baseline']['avg_calls_per_example']:.2f}")

print("\nPROPOSED METHOD:")
print(f"  Fusion rate:     {metrics['proposed']['fusion_rate']:.1%}")
print(f"  Fission rate:    {metrics['proposed']['fission_rate']:.1%}")
print(f"  Error rate:      {metrics['proposed']['error_rate']:.1%}")
print(f"  Total API calls: {metrics['proposed']['api_calls']}")
print(f"  Avg calls/example: {metrics['proposed']['avg_calls_per_example']:.2f}")

print("\nIMPROVEMENT:")
print(f"  API reduction:   {metrics['improvement']['api_reduction_pct']:.1f}%")
print(f"  Error rate diff: {metrics['improvement']['error_rate_diff']:+.1%}")

# Save results (optional - simulating the original script's output)
print(f"\nAPI reduction: {metrics['improvement']['api_reduction_pct']:.1f}%")

## Raw Metrics Output

Here's the complete metrics dictionary (equivalent to what would be saved to `eval_out.json`):

In [ ]:
# Display the complete metrics as JSON (equivalent to eval_out.json)
print(json.dumps(metrics, indent=2))

## Conclusion

This notebook demonstrates the evaluation of the DKW Controller methods. Key findings:

1. **API Efficiency**: The proposed method achieves a **32.5% reduction** in API calls compared to the baseline
2. **Decision Strategy**: The proposed method uses a mix of fusion (65%) and fission (35%) decisions, while the baseline only uses fission
3. **Trade-off**: There's a slight increase in error rate (1 percentage point) but significant API savings

### Next Steps

You can modify this notebook to:
- Test with different sample data
- Adjust the decision thresholds
- Add visualization of the results
- Experiment with different metrics calculations